<a href="https://colab.research.google.com/github/Alla-ud-din/Classes-Q3-Q4/blob/master/LangGraph/AGENTIC_PROJECTS/00_chat_agent_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Requirement Document: Agentic AI Chatbot Prototype 1**

---

#### **Objective:**
Create a chatbot prototype that talks to users, remembers what they say during conversation, and helps them solve problems.

---

#### **Project Requirements:**
1. **Choose Your Niche:** Pick a specific group of people or industry to help with your chatbot (e.g., students, healthcare, customer service).

2. **Make Smart Conversations:** Make sure the chatbot answers correctly and understands what users want. Use Prompt Engineering.

3. **Add Useful Tools:** Give the chatbot features like helping users submit complaints, giving outfit ideas, or making study plans.

4. **Short-Term Memory:** Make the chatbot remember the current conversation to give better answers.

5. **Use Google Colab:** Build and test your chatbot using this platform.

---

#### **Deliverables:**
- A working chatbot prototype - Google Collab Link.
- A list of tools and examples of how the chatbot can help people.

---

## **Submission Form**

Submit your projects here:  
[**Project Submission Form**](https://forms.gle/yB6X4TzE2dCVThCj8)

---

#### **User Stories:**
- **As a student,** I want the chatbot to help me research & plan my study schedule, so I can prepare well for exams.
- **As a customer,** I want the chatbot to help me file a complaint, so I can get my problem fixed quickly.
- **As a partygoer,** I want the chatbot to give me outfit ideas, so I feel good and confident at the event.
- **As a healthcare user,** I want the chatbot to answer simple health questions, so I can take care of myself better.

---

#### **Competencies/Outcomes:**
By completing this project, students will:

1. **Basic Level - REACT Architecture:** Understand how to build an AI chatbot prototype.
2. **Basic Level - Prompt Engineering:** Learn how to use prompt engineering for smart and relevant conversations.
3. **Basic Level - Tool Calling & Chat Management:** Understand and demonstrating Tool Calling using LLMs.
4. **Basic Level - Short-Term Memory:** Manage Chat Conversation s improve user interactions.

---

In [ ]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_google_genai tavily-python langchain_community

In [ ]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=GEMINI_API_KEY
)

from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=2)
# llm.invoke("greet me")
# tavily_search.invoke("What's a 'node' in LangGraph?")

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage, SystemMessage
from IPython.display import Image, display, Markdown
from langgraph.graph.state import CompiledStateGraph
from langgraph.checkpoint.memory import MemorySaver

# Define the State class to manage the state transitions
class State(TypedDict):
    user_query: Annotated[list[AnyMessage], add_messages]  # Stores user input messages
    response: Annotated[list[AnyMessage], add_messages]  # Stores assistant responses

# Initialize a state graph builder
graph_builder = StateGraph(State)

# Define tool: Health Tips
def health_tips(state: State) -> State:
    """Provide health tips using Tavily Search."""
    user_query = state['user_query'][-1].content  # Extract user query
    tavily_response = tavily_search.invoke(user_query)  # Call Tavily Search API
    print("Here are some health tips: \n")
    return {"response": [AIMessage(content=tavily_response)]}  # Return the response

# Define tool: Medical FAQs
def medical_faqs(state: State) -> State:
    """Provide medical FAQs for a given condition using Tavily Search."""
    user_query = state['user_query'][-1].content  # Extract user query
    query = f"Medical FAQs about: {user_query}"
    tavily_response = tavily_search.invoke(query)  # Call Tavily Search API
    print("Here are some FAQs: \n")
    return {"response": [AIMessage(content=tavily_response)]}  # Return the response

# Define tool: Medication Info
def medication_info(state: State) -> State:
    """Provide information about medications."""
    user_query = state['user_query'][-1].content  # Extract user query
    query = f"Medication information for: {user_query}"
    tavily_response = tavily_search.invoke(query)  # Call Tavily Search API
    print("Here is some information about the medication:\n")
    return {"response": [AIMessage(content=tavily_response)]}  # Return the response

# Define tool: General Query Handler
def general_query_handler(state: State) -> State:
    """Handle general queries unrelated to medical topics."""
    user_query = state['user_query'][-1].content  # Extract user query
    response = f"It seems like your query is unrelated to medical topics. Here's a general response to '{user_query}': I'm here to help, but I specialize in health-related queries. Could you clarify or ask something health-related?"
    return {"response": [AIMessage(content=response)]}  # Return the response

# List of tools
tools = [health_tips, medical_faqs, medication_info, general_query_handler]

# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

# Define a system message to guide the assistant
sys_msg = SystemMessage(content="You are a helpful assistant tasked with helping user with their medical queries. Return tool call if user's query is related to any tool mentioned")

# Define the assistant node
def assistant(state: State) -> State:
    response = llm_with_tools.invoke([sys_msg] + state["user_query"])  # Get response from the LLM
    return {"response": [response]}  # Return the response

# Define tool calling logic
def tool_calling(state: State):
    """
    Determine the next tool to call based on model responses.
    """
    try:
        if state['response'] and hasattr(state['response'][-1], 'tool_calls') and len(state['response'][-1].tool_calls) > 0:
            tool_name = state['response'][-1].tool_calls[-1].get("name", END)  # Extract tool name
            return tool_name
        return END  # End if no tool calls are present

    except Exception as e:
        print(f"[ERROR in tool_calling]: {e}")
        return END  # End in case of error

# Initialize the state graph builder
builder: StateGraph = StateGraph(State)

# Define nodes in the graph
builder.add_node("assistant", assistant)
builder.add_node("health_tips", health_tips)
builder.add_node("medical_faqs", medical_faqs)
builder.add_node("medication_info", medication_info)
builder.add_node("general_query_handler", general_query_handler)

# Define edges in the graph
builder.add_edge(START, "assistant")  # Start with the assistant node
builder.add_conditional_edges("assistant", tool_calling)  # Add conditional edges for tool calling
builder.add_edge("health_tips", END)  # End after health tips
builder.add_edge("medical_faqs", END)  # End after medical FAQs
builder.add_edge("medication_info", END)  # End after medication info
builder.add_edge("general_query_handler", END)  # End after general query handling

# To save consersation in memory
memory: MemorySaver = MemorySaver()
# Compile the state graph
graph: CompiledStateGraph = builder.compile(checkpointer=memory)

# Visualize the graph
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))




In [ ]:
# Specify a thread configuration
config = {"configurable": {"thread_id": "1"}}

In [ ]:
# Main interaction loop
while True:
    try:
        # Get user input
        user_input = input("Welcome to HealthHub! How can I assist you today? (type 'quit', 'exit', 'q' to exit): ")

        # Exit loop if the user types 'quit', 'exit', or 'q'
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye! Stay healthy and take care!")
            break  # Exit loop if user wants to quit

        # Invoke the workflow graph with the user's query
        result = graph.invoke({"user_query": user_input}, config)

        # Get the most recent message from the response
        last_message = result["response"][-1]

        # Check if the message has 'content' and it is not empty
        if hasattr(last_message, 'content') and last_message.content:
            # Iterate over each response in the 'content' list
            for response in last_message.content:
                # Print the response in markdown format
                print(f"**HealthHub Response:**\n")
                print(f"### [Link to source]({response['url']})\n")  # Display the URL as a markdown link
                print(f"**Content:**\n{response['content']}\n")  # Display the content

    except Exception as e:
        # Catch any exceptions and print an error message
        print(f"[ERROR in main loop]: {e}")
        print("Bot: Sorry, something went wrong. Please try again.")


In [ ]:
snapshot = graph.get_state(config)
snapshot.values['user_query']

In [ ]:
snapshot.values['response'][-1].content